In [ ]:
import pandas as pd
import folium
import geopandas as gpd
from matplotlib import cm, colors
from sklearn.preprocessing import MinMaxScaler
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np

Heat map of:
- Bedroom distribution
- Bathroom distribution
- Liveable distribution
- Affordable distribution
- Growth distribution

In [ ]:
MOST_IMPORTANT_FEATURES = ['bathrooms', 'bedrooms']

MOST_LIVEABLE = {'CANTEBURY': 3126, 'HAWTHORN EAST': 3123, 'ST KILDA EAST': 3183, 'BALACLAVA': 3183, 'CAULFIELD SOUTH': 3162, 
                'CAULFIELD': 3162, 'FITZROY NORTH': 3068, 'CLIFTON HILL': 3068, 'ELSTERNWICK': 3185, 'RIPPONLEA': 3185} # Clifton hill 3066, 3067, 3068

MOST_AFFORDABLE = {'NYORA': 3987, 'ST ANDREWS': 3761, 'STRATHMERTON': 3641, 'BELGRAVE': 3160, 'GORDON': 3345, 'WHOROULY': 3735, 
                    'NEW GISBOURNE': 3438, 'SHELFORD': 3329, 'SILVAN': 3795, 'ALPHINGTON': 3078}

MOST_GROWTH = {'MILDURA': 3500, 'SEAFORD-CARRUM DOWNS': 3201, 'BERWICK': 3806, 'SHEPPARTON': 3630, 'CRANBOURNE': 3977}


Listings

In [ ]:
listings = pd.read_csv('../data/curated/cleaned_real_estate_data.csv')
listings = listings[['postcode', 'weekly_rent'] + MOST_IMPORTANT_FEATURES + ['lat', 'lon']]
listings["rooms"] = listings["bathrooms"] + listings["bedrooms"]
scaler = MinMaxScaler()
scaled = scaler.fit_transform(listings[["rooms"]])
listings["rooms_rank"] = scaled + 0.3
listings

Liveability and affordability

In [ ]:
postcode_la = pd.read_csv('../data/outputs/postcode_livability_affordability.csv')
postcode_la = postcode_la[['postcode', 'livability_score', 'affordability_score', 'lat', 'lon']]
postcode_la['livability_score'] = MinMaxScaler().fit_transform(postcode_la[['livability_score']])
postcode_la['affordability_score'] = MinMaxScaler().fit_transform(postcode_la[['affordability_score']])

postcode_la['livability_rank'] = postcode_la['livability_score'].rank(ascending=False)
postcode_la['affordability_rank'] = postcode_la['affordability_score'].rank(ascending=False)

postcode_la['livability_rank_scaled'] = 1 - MinMaxScaler().fit_transform(postcode_la[['livability_rank']])
postcode_la['affordability_rank_scaled'] = 1 - MinMaxScaler().fit_transform(postcode_la[['affordability_rank']])

# postcode_la['merged_score'] = 0.75*postcode_la['livability_score'] + 0.25*postcode_la['affordability_score'] # most of affordable is rural anyway so weight less
# postcode_la['merged_rank'] = postcode_la['merged_score'].rank(ascending=False)

# scaler = MinMaxScaler()
# scaled = scaler.fit_transform(postcode_la[['merged_rank']])
# postcode_la['merged_rank_scaled'] = 1 - scaled
postcode_la

Growth

In [ ]:
suburb_growth = pd.read_csv('../data/outputs/overall_growth.csv').drop(columns=['Unnamed: 0']).rename(columns={'Suburb': 'suburb'})
growth_loc = pd.read_csv('../data/raw/growth_suburbs_latlon.csv')


suburb_growth = suburb_growth[suburb_growth['suburb'] != "AVERAGE"]
suburb_growth = suburb_growth.merge(growth_loc, on='suburb', how='left')
suburb_growth['price_change_std'] = MinMaxScaler().fit_transform(suburb_growth[['Price_change (%)']])

suburb_growth["growth_rank"] = suburb_growth["Price_change (%)"].rank(ascending=False)
scaler = MinMaxScaler()
scaled = scaler.fit_transform(suburb_growth[["growth_rank"]])
suburb_growth["growth_rank_scaled"] = 1 - scaled

suburb_growth

Building geovisualisation

In [ ]:
agg = postcode_la

In [ ]:
# Convert to GeoDataFrame
agg_gdf = gpd.GeoDataFrame(
    agg,
    geometry=gpd.points_from_xy(agg['lon'], agg['lat']),
    crs="EPSG:4326"
)

listings_gdf = gpd.GeoDataFrame(
    listings,
    geometry=gpd.points_from_xy(listings['lon'], listings['lat']),
    crs="EPSG:4326"
)

growth_gdf = gpd.GeoDataFrame(
    suburb_growth,
    geometry=gpd.points_from_xy(suburb_growth['lon'], suburb_growth['lat']),
    crs="EPSG:4326"
)


In [ ]:
import folium
from matplotlib import cm, colors
import numpy as np
from shapely.geometry import shape

def create_polygon_heatmap_layer(
    polygons_gdf,         # GeoDataFrame with polygon geometries
    points_gdf=None,      # Optional: points to aggregate inside polygons (spatial join)
    layer_name="Heatmap",
    color_col=None,       # Column in polygons_gdf to color by (if no points_gdf)
    agg_func=np.mean,     # Aggregation function if points_gdf is used
    colormap_name="YlGn",
    tooltip_cols=None
):
    """
    Creates a Folium FeatureGroup for polygon heatmaps.
    
    If points_gdf is provided, color each polygon by aggregating values of points inside it.
    Otherwise, use the column color_col in polygons_gdf.
    """
    layer = folium.FeatureGroup(name=layer_name, show=True)
    cmap = cm.get_cmap(colormap_name)

    for _, poly_row in polygons_gdf.iterrows():
        # Determine value to color by
        if points_gdf is not None:
            points_inside = points_gdf[points_gdf.geometry.within(poly_row.geometry)]
            if points_inside.empty:
                value = np.nan
            else:
                value = agg_func(points_inside[color_col])
        else:
            value = poly_row[color_col] if color_col in poly_row else np.nan

        # Style dictionary
        if np.isnan(value):
            style = {'fillOpacity': 0, 'weight': 0.5, 'color': 'grey'}
        else:
            style = {
                'fillColor': colors.to_hex(cmap(value)),
                'color': 'black',
                'weight': 0.5,
                'fillOpacity': 0.7
            }

        # Tooltip
        tooltip_html = ""
        if tooltip_cols:
            tooltip_html = "<br>".join([f"{col}: {poly_row[col]}" for col in tooltip_cols])

        folium.GeoJson(
            poly_row.geometry,
            style_function=lambda x, s=style: s,
            tooltip=tooltip_html
        ).add_to(layer)

    return layer

In [ ]:
postcode_gdf = gpd.read_file('../data/raw/SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp')

In [ ]:
postcode_gdf = postcode_gdf[postcode_gdf.geometry.notnull() & postcode_gdf.is_valid]
agg_gdf = agg_gdf[agg_gdf.geometry.notnull() & agg_gdf.is_valid]
listings_gdf = listings_gdf[listings_gdf.geometry.notnull() & listings_gdf.is_valid]
growth_gdf = growth_gdf[growth_gdf.geometry.notnull() & growth_gdf.is_valid]

In [ ]:
# Create map centered on Victoria
vic_center = [listings['lat'].mean(), listings['lon'].mean()]
m = folium.Map(location=vic_center, zoom_start=8, tiles='cartodbpositron')

In [ ]:
# Add SA2 boundaries
folium.GeoJson(
    postcode_gdf,
    name='SA2 Boundaries',
    style_function=lambda feature: {
        'fillColor': 'none',   # transparent fill
        'color': 'black',        # boundary color
        'weight': 1,
        'dashArray': '2, 2',
        'opacity': 0.6
    },
    tooltip=folium.GeoJsonTooltip(fields=['SA2_NAME21'], aliases=['SA2:'])
).add_to(m)

In [ ]:
# Layer: Bathrooms + Bedrooms (Red)
room_layer = folium.FeatureGroup(name='Bathrooms and Bedrooms (Red)', show=True)

for _, row in listings.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=row["rooms_rank"] * 1.5,
        color=None,
        fill=True,
        fill_color='rgba(255, 0, 0, 0.4)',
        fill_opacity=0.4
    ).add_to(room_layer)

In [ ]:
# Layer: Forecasted Growth (Purple)
growth_layer = folium.FeatureGroup(name='Forecasted Growth (Purple)', show=True)
cmap_merged = cm.get_cmap('Purples')
for _, row in suburb_growth.iterrows():
    hexcol = colors.to_hex(cmap_merged(row['price_change_std']))
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=(row['growth_rank_scaled'] + 0.3) * 10,
        fill=True,
        fill_color=hexcol,
        fill_opacity=row['growth_rank_scaled'],
        color=None,
        tooltip=f"Location: {row['suburb']}<br>Price Change (%): {row['Price_change (%)']:.3f}<br>Rank: {int(row['growth_rank'])}"
    ).add_to(growth_layer)

In [ ]:
# # Layer: Merged Livability & Affordability (Green)
# merged_layer = folium.FeatureGroup(name='Livability & Affordability (Green)', show=True)
# cmap_merged = cm.get_cmap('YlGn')

# for _, row in agg.iterrows():
#     hexcol = colors.to_hex(cmap_merged(row['merged_score']))
#     folium.CircleMarker(
#         location=[row['lat'], row['lon']],
#         radius=(row['merged_rank_scaled'] + 0.3) * 5,
#         fill=True,
#         fill_color=hexcol,
#         fill_opacity=row['merged_rank_scaled'],
#         color=None,
#         tooltip=f"Postcode: {row['postcode']}<br>Liveability and Affordability Index: {row['merged_score']:.3f}<br>Rank: {int(row['merged_rank'])}"
#     ).add_to(merged_layer)

In [ ]:
# Layer: Livability (Blue)
livability_layer = folium.FeatureGroup(name='Livability (Blue)', show=True)
cmap_merged = cm.get_cmap('Blues')

for _, row in agg.iterrows():
    hexcol = colors.to_hex(cmap_merged(row['livability_score']))
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=(row['livability_rank_scaled'] + 0.3) * 5,
        fill=True,
        fill_color=hexcol,
        fill_opacity=row['livability_rank_scaled'],
        color=None,
        tooltip=f"Postcode: {row['postcode']}<br>Liveability Index: {row['livability_score']:.3f}<br>Rank: {int(row['livability_rank'])}"
    ).add_to(livability_layer)

In [ ]:
# Layer: Affordability (Green)
affordability_layer = folium.FeatureGroup(name='Affordability (Green)', show=True)
cmap_merged = cm.get_cmap('Greens')

for _, row in agg.iterrows():
    hexcol = colors.to_hex(cmap_merged(row['affordability_score']))
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=(row['affordability_rank_scaled'] + 0.3) * 5,
        fill=True,
        fill_color=hexcol,
        fill_opacity=row['affordability_rank_scaled'],
        color=None,
        tooltip=f"Postcode: {row['postcode']}<br>Affordability Index: {row['affordability_score']:.3f}<br>Rank: {int(row['affordability_rank'])}"
    ).add_to(affordability_layer)

In [ ]:
# # Layer: Merged Livability & Affordability (Green)
# merged_layer = create_polygon_heatmap_layer(
#     polygons_gdf=postcode_gdf,
#     points_gdf=agg_gdf,
#     layer_name="Livability & Affordability (Green)",
#     color_col="merged_score",
#     agg_func=np.mean,
#     colormap_name="YlGn",
#     tooltip={
#         'postcode': 'Postcode',
#         'merged_score': 'Liveability & Affordability Index',
#         'merged_rank': 'Rank'
#     }
# )

In [ ]:
# # Layer: Forecasted Growth (Blue)
# growth_layer = create_polygon_heatmap_layer(
#     polygons_gdf=growth_gdf,
#     points_gdf=listings_gdf,
#     layer_name="Forecasted Growth (Blue)",
#     color_col="price_change_std",
#     agg_func=np.mean,
#     colormap_name="Blues",
#     tooltip={
#         'suburb': 'Location',
#         'Price_change (%)': 'Price Change (%)',
#         'growth_rank': 'Rank'
#     }
# )

In [ ]:
# # Layer: Bathrooms (red)
# offset = 0.0001  # small offset to avoid overlap
# bathroom_layer = folium.FeatureGroup(name='Bathrooms (Red)', show=True)

# for _, row in listings.iterrows():
#     folium.CircleMarker(
#         location=[row['lat'] + offset, row['lon']],
#         radius=2,
#         color=None,
#         fill=True,
#         fill_color=f'rgba({int(row["bathrooms"] * 50)}, 0, 0, 0.6)',
#         fill_opacity=0.6
#     ).add_to(bathroom_layer)

In [ ]:
# # Layer: Bedrooms (blue)
# bedroom_layer = folium.FeatureGroup(name='Bedrooms (Blue)', show=True)

# for _, row in listings.iterrows():
#     folium.CircleMarker(
#         location=[row['lat'] - offset, row['lon']],
#         radius=2,
#         color=None,
#         fill=True,
#         fill_color=f'rgba(0, 0, {int(row["bedrooms"] * 50)}, 0.6)',
#         fill_opacity=0.6
#     ).add_to(bedroom_layer)

In [ ]:
growth_layer.add_to(m)
livability_layer.add_to(m)
affordability_layer.add_to(m)
room_layer.add_to(m)
# bathroom_layer.add_to(m)
# bedroom_layer.add_to(m

folium.LayerControl().add_to(m)
title_html = '''
             <h3 align="center" style="font-size:20px"><b>Victorian Map of Suburbs: Predicted Growth (Purple), Livability (Blue), Affordability (Green), and Important Rental Predictors (Red)</b></h3>
             '''
m.get_root().html.add_child(folium.Element(title_html))

m.save("map.html")
m